In [34]:
import pennylane as qml
import yfinance as yf
import pandas as pd
import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [48]:
# Import Data

ticker = "TSLA"
start = "2020-01-01"
end = "2025-01-01"
interval = "1d"
stock_data = yf.download(ticker, start, end, interval, auto_adjust = False)

[*********************100%***********************]  1 of 1 completed


In [49]:
stock_data

Price,Adj Close,Close,Dividends,High,Low,Open,Stock Splits,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA
Date,,,,,,,,
2020-01-02,28.684000,28.684000,0.0,28.713333,28.114000,28.299999,0.0,142981500
2020-01-03,29.534000,29.534000,0.0,30.266666,29.128000,29.366667,0.0,266677500
2020-01-06,30.102667,30.102667,0.0,30.104000,29.333332,29.364668,0.0,151995000
2020-01-07,31.270666,31.270666,0.0,31.441999,30.224001,30.760000,0.0,268231500
2020-01-08,32.809334,32.809334,0.0,33.232666,31.215334,31.580000,0.0,467164500
...,...,...,...,...,...,...,...,...
2024-12-24,462.279999,462.279999,0.0,462.779999,435.140015,435.899994,0.0,59551800
2024-12-26,454.130005,454.130005,0.0,465.329987,451.019989,465.160004,0.0,76366400


In [36]:
# Feature Selection

stock_features = stock_data[["Open", "High", "Low", "Adj Close", "Volume"]].copy()

stock_features["Log Returns"] = np.log(stock_features["Adj Close"]/stock_features["Adj Close"].shift(1))
stock_features["Price Change"] = stock_features["Adj Close"] - stock_features["Adj Close"].shift(1)

# Classifieres 
classifiers = pd.DataFrame()
classifiers['target'] = (stock_features['Adj Close'].shift(-1) > stock_features['Adj Close']).astype(int)


# Drop Na
stock_features = stock_features.dropna()
classifiers = classifiers.loc[stock_features.index]



# Scale Features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(stock_features)

classifier_array = classifiers.to_numpy().flatten()





In [37]:
classifier_array

array([1, 1, 1, ..., 0, 0, 0])

In [38]:
x_train, x_test, y_train, y_test = train_test_split(features_scaled, classifier_array, test_size = .2)

In [39]:
import pennylane as qml
from pennylane.kernels import square_kernel_matrix
from pennylane.kernels import kernel_matrix
from sklearn.svm import SVC

In [40]:
# Create Qubits
n_qubits = x_train.shape[1] # 
dev = qml.device("default.qubit", wires=n_qubits) # We are simulating a quantum system with pennylanes default

Note: x_tain.shape puts the shape othe 2d array as a tuple with the first entry being the rows and the second being the columns. By grabbing the number of columns, we are automatically grabbing the numbers of features! For quantum machine learning each feature is encoded in sperate qubit!

In [41]:
# Angle Encoding Circut
@qml.qnode(dev)
def circut(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits)) # template that applies rotation aroundy axis by default!
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits)) # calculates the hermitian adjoint (inverse)
    return qml.probs(wires=range(n_qubits))


We use angle encoding to convert the classic data into quantum states but creating a su2 rotation from the original data and applying it into the original state. 

In [42]:
from pennylane.kernels import square_kernel_matrix
from pennylane.kernels import kernel_matrix

# Create Kernel Funnction with quantum circut
def quantum_kernel(x1, x2):
    return circut(x1, x2)[0]


# Kernal Matices
x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

In [43]:
# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)


In [44]:
from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.5238095238095238


In [45]:
# Optimization: Introduce Entaglement in Feature Maps (CNOT Entaglement)

@qml.qnode(dev)
def circut(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))

    # CNOT ENTAGLEMENT
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits)) 
    return qml.probs(wires=range(n_qubits))


# Create Kernel Funnction with quantum circut
def quantum_kernel(x1, x2):
    return circut(x1, x2)[0]


# Kernal Matices
x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)

from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))



Accuracy: 0.5238095238095238


In [46]:
# Optimization: Introduce Entaglement in Feature Maps (XX Entaglement)

@qml.qnode(dev)
def circuit(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))
    
    # Ising Entaglment
    for i in range(n_qubits - 1):
        qml.IsingXX(np.pi / 2, wires=[i, i + 1])
    
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))


# Create Kernel Funnction with quantum circut
def quantum_kernel(x1, x2):
    return circut(x1, x2)[0]


# Kernal Matices
x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)

from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.5238095238095238


In [47]:
# Optimization With Layers

@qml.qnode(dev)
def circuit(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))
    
    # Ising entanglement between neighbors
    for _ in range(n_layers):  
        for i in range(n_qubits - 1):
            qml.IsingXX(np.pi / 2, wires=[i, i + 1])
        for i in range(n_qubits):
            qml.RY(np.pi / 4, wires=i)  
    
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))


# Create Kernel Funnction with quantum circut
def quantum_kernel(x1, x2):
    return circut(x1, x2)[0]


# Kernal Matices
x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)

from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.5238095238095238
